In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
 # Azure storage access info
blob_account_name = "bgupb202402juanbarriento"
blob_container_name = "marketplace"
blob_sas_token = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-12-01T01:56:16Z&st=2024-10-16T17:56:16Z&spr=https&sig=cM7vK4vvrL9YH31ZYI%2BX%2BmssMUTdRieP%2BMGcVNPoagA%3D'

# Allow SPARK to read from Blob remotely
wasbs_path = f'wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/'
spark.conf.set( f'fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net' , blob_sas_token)
print('Remote blob path: ' + wasbs_path)

Remote blob path: wasbs://marketplace@bgupb202402juanbarriento.blob.core.windows.net/


# #Raw Files

In [0]:
display(dbutils.fs.ls("dbfs:/bigdata202402/marketplace/jdbr/raw"))

path,name,size,modificationTime
dbfs:/bigdata202402/marketplace/jdbr/raw/2019-Dec.csv.gz,2019-Dec.csv.gz,2951549375,1728325030000
dbfs:/bigdata202402/marketplace/jdbr/raw/2019-Nov.csv.gz,2019-Nov.csv.gz,2890421023,1728325204000
dbfs:/bigdata202402/marketplace/jdbr/raw/2019-Oct.csv.gz,2019-Oct.csv.gz,1741928540,1728324857000
dbfs:/bigdata202402/marketplace/jdbr/raw/2020-Apr.csv.gz,2020-Apr.csv.gz,2929933389,1728326221000
dbfs:/bigdata202402/marketplace/jdbr/raw/2020-Feb.csv.gz,2020-Feb.csv.gz,2347087112,1728325984000
dbfs:/bigdata202402/marketplace/jdbr/raw/2020-Jan.csv.gz,2020-Jan.csv.gz,2390063923,1728325496000
dbfs:/bigdata202402/marketplace/jdbr/raw/2020-Mar.csv.gz,2020-Mar.csv.gz,2417362205,1728326101000


In [0]:
df_raw = spark.read.format("csv").option("header", "true").load("/bigdata202402/marketplace/jdbr/raw")

In [0]:
display(df_raw)

In [0]:
df_filtered = df_raw.filter((df_raw.category_code != 'null') & (df_raw.brand != 'null') & (df_raw.user_session != 'null'))

In [0]:
df_filtered_categories = df_filtered.withColumn('category', split(col('category_code'), '\.')
.getItem(0)).withColumn('subcategory', split(col('category_code'), '\.').getItem(1))

In [0]:
display(df_filtered_categories) 

In [0]:
df_bronze = df_filtered_categories.withColumn(
'price'
, df_filtered_categories[
'price'
].cast(
'double'
)).withColumn(
'month'
, month(
'event_time'
)).withColumn(
'year'
,year(
'event_time'
)).withColumn(
'event_time_timestamp'
, to_timestamp(
'event_time'
))

In [0]:
#Delete any residual data from prior executions for an idempotent run
dbutils.fs.rm(f"{wasbs_path}/bronze",recurse=True)

True

In [0]:

df_bronze.write.format("delta").mode("overwrite").partitionBy("year", "month").save(f"{wasbs_path}/bronze")

In [0]:
%sql
USE jdbr_mkp;
DROP TABLE IF EXISTS mkp_bronze;
CREATE TABLE mkp_bronze
USING DELTA
LOCATION 'wasbs://marketplace@bgupb202402juanbarriento.blob.core.windows.net/bronze/'

In [0]:
%sql
USE jdbr_mkp;
select * from mkp_bronze
LIMIT 100;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2944297148578667>, line 1
----> 1 df_bronze.write.format("delta").mode("overwrite").partitionBy("year", "month").save(f"{wasbs_path}/bronze")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1732, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1730     self._jwrite.save()
   1731 else:
-> 1732     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__

## Bronze Layer